In [3]:
import os
from PIL import Image
import torch
from torchvision import transforms
from tqdm.notebook import tqdm

In [4]:
%run ./covid_kaggle.py

In [5]:
2327 * 100 / 80

2908.75

In [3]:
dataset = CovidKaggleDataset(dataset_type='train')
len(dataset)

2327

### Calculate mean and std

In [7]:
to_tensor = transforms.ToTensor()

In [14]:
summed = torch.zeros(3, 1024, 1024)

for image_name, label in tqdm(dataset.images_metadata):
    image_path = os.path.join(DATASET_DIR, image_name)
    image = to_tensor(Image.open(image_path).convert('RGB'))
    
    summed += image

summed /= len(dataset.images_metadata)

In [15]:
summed.mean(dim=-1).mean(dim=-1)

tensor([0.4872, 0.4875, 0.4876])

In [16]:
summed.std(dim=-1).std(dim=-1)

tensor([0.0352, 0.0352, 0.0352])

In [18]:
image.mean(), image.std()

(tensor(0.5613), tensor(0.2393))

### Compute amounts

In [17]:
from collections import defaultdict

In [10]:
def compute_amounts(dataset):
    amounts = dict()
    for label in dataset.labels:
        amount = sum(presence for idx, presence in dataset.get_labels_presence_for(label))
        amounts[label] = amount
    
    return amounts

In [12]:
train_dataset = CovidKaggleDataset(dataset_type='train')
val_dataset = CovidKaggleDataset(dataset_type='val')
test_dataset = CovidKaggleDataset(dataset_type='test')

In [16]:
train_amounts = compute_amounts(train_dataset)
val_amounts = compute_amounts(val_dataset)
test_amounts = compute_amounts(test_dataset)

In [18]:
total_amounts = defaultdict(lambda: 0)
for am in [train_amounts, val_amounts, test_amounts]:
    for label, value in am.items():
        total_amounts[label] += value
        
total_amounts

defaultdict(<function __main__.<lambda>()>,
            {'covid': 219, 'pneumonia': 1345, 'normal': 1341})

In [20]:
total = sum(total_amounts.values())
total

2905

In [22]:
for label, value in total_amounts.items():
    print(label, value/total*100)

covid 7.538726333907057
pneumonia 46.29948364888124
normal 46.1617900172117
